In [ ]:
# Import necessary libraries
import openai
import requests
import os
import json
import random
from concurrent.futures import ThreadPoolExecutor

# Load API key from environment variable stored in the .env file at the root directory
api_key = os.getenv("OPENAI_API_KEY")

# Configuration for various models provided by OpenAI
model_config = {
    'chat_model': {
        "options": ["gpt-3.5-turbo-0125", "gpt-4-turbo-2024-04-09"],  # Available model options for chat
        "selected": 1  
    },
    'audio_model': {
        "options": ["tts-1", "tts-1-hd"],  # Text-to-speech model versions
        "selected": 1 
    },
    'image_model': {
        "options": ["dall-e-2", "dall-e-3"],  # Options for image generation models
        "selected": 1 
    }
}

def initialize_openai():
    """
    Initialize the OpenAI client using the configured API key.
    
    Returns:
        openai.OpenAI: Configured client instance ready for making API requests.
    """
    return openai.OpenAI(api_key=api_key)

# Initialize the OpenAI client and store it in 'client'
client = initialize_openai()


In [ ]:
def select_random_genre() -> dict:
    """
    Selects a random genre and provides details including the initial scene description,
    associated writer, artist, and other relevant metadata from predefined options.

    Returns:
        dict: A dictionary containing the selected genre's name, writer, artist,
              initial scene description, and additional metadata such as voice and time.
    """
    # Data structure containing genre options and their associated details
    genres_data = {
        "genres": [
            {
                "name": "Adventure",
                "writer": "Robert Louis Stevenson",
                "artist": "A Renaissance painter of great renown",
                "initial_scene_description": "Amidst the verdant embrace of a tropical island, you stand at the threshold of an ancient path that twists eastward beneath the towering canopy. Every leaf and vine is painted with the vivid hues that only the stroke of nature, and perhaps a Renaissance master, could conceive. This path, carpeted with fallen blossoms and the detritus of many rains, beckons you deeper into its secrets and away from the sound of the distant surf.",
                "voice": "onyx",
                "time": "6:45 AM"
            },
            {
                "name": "Horror",
                "writer": "Edgar Allan Poe",
                "artist": "a daguerreotype photographer of great renown",
                "initial_scene_description": "The chill of an overcast day in a deserted seaside town grips your bones as you find yourself at the mouth of a narrow alley. This alley, like a forgotten vein of the old town, runs north into the heart of an encroaching fog that seems almost alive. The monochrome hues of the abandoned shops and homes around you suggest the silvered daguerreotypes of a time long past, each window and shadow capturing the essence of a ghost story waiting to be told.",
                "voice": "shimmer",
                "time": "7:00 PM"
            },
            {
                "name": "Science Fiction",
                "writer": "Neal Stephenson",
                "artist": "H. R. Giger",
                "initial_scene_description": "You find yourself standing at the edge of a vast, desolate plaza. The skeletons of towering skyscrapers loom like sentinels around the perimeter, their once-gleaming facades now dulled and pitted by the relentless passage of time. To the east, a monolithic structure's shattered glass creates a mosaic of reflections from the weak, diffused sunlight filtering through the dense overcast sky. A weathered road, cracked and overgrown with resilient flora, offers a path eastward, winding through the heart of this ghostly metropolis.",
                "voice": "onyx",
                "time": "12:00 PM"
            }
        ]
    }

    # Randomly select one genre from the list
    random_genre = random.choice(genres_data["genres"])

    # Prepare the selected genre data for output
    story_data = {
        'name': random_genre["name"],
        'writer': random_genre["writer"],
        'artist': random_genre["artist"],
        'initial_scene_description': random_genre["initial_scene_description"],
        'time': random_genre["time"],
        'voice': random_genre["voice"],
        'previous_scene': '',
        'player_move': '',
    }

    # Print formatted story data to console (for debugging purposes or user feedback)
    print(json.dumps(story_data, indent=4))

    return story_data


In [ ]:
def generate_prompt(story_data: dict) -> list:
    """
    Generates a detailed prompt for a client-based on the given story data for a text adventure game. The prompt
    includes conditions for scene progression and options based on the game's narrative flow.
    
    Args:
        story_data (dict): A dictionary containing details about the selected genre, including the writer, genre name,
                           initial scene description, and additional metadata such as time and previously available actions.
    
    Returns:
        list: A list containing a single dictionary which forms a detailed prompt structured for further processing,
              particularly aimed at constructing narrative and visual content in a game setting.
    """
    # Check if there is a previous scene to reference; adapt the scene description and back option accordingly
    if story_data['previous_scene'].strip():
        scene_description = f"a previous game scene which read: '{story_data['previous_scene']}' and the player chose to move {story_data['player_move']}."
        back_option = "Additionally, include an option to 'go back', allowing the player to return to the previous location."
    else:
        scene_description = story_data['initial_scene_description']
        back_option = "This scene should not offer the player a 'go back' option, as it is the starting location of the game."
    
    # Create the foundational part of the prompt detailing the writer's role and the narrative context
    foundation_prompt = (
        f"You are {story_data['writer']}, writing a {story_data['name']} text game in your literary style for a twelve-year-old. "
        f"The scene to write is based on {scene_description} at {story_data['time']}. "
        "Do not name the player or give them any tools or weapons. Please return a detailed JSON object with keys for 'story', "
        "'scene_description', and 'player_options'."
    )

    # Define the structure and constraints of the narrative to be generated
    data_shape_prompt = (
        "The scene description should be used by an image generator to create a visualization of the story from the player's "
        "first-person perspective. It must be detailed and vivid, always feature at least one path for the player to move in, "
        "and be under 500 characters. The player options should only include directions explicitly mentioned in the scene description "
        "(e.g., if a pathway is described as leading forward, 'forward' should be the only option). "
        f"{back_option} 'Player_options' should be structured as an object with a 'direction' key which should be an array containing "
        "the direction and a 'command text' like ['forward', 'Continue down alley'], limited to three words maximum, and a 'transition_text' key. "
        "The transition_text should describe the action of moving in that direction, keeping these texts to one or two sentences. "
        "The JSON object structure must include 'story', 'scene_description', and 'player_options', where 'player_options' contain 'directions' "
        "with all available directions the player can move, each including 'direction', 'command_text', and 'transition_text'."
    )

    # Construct the full client prompt
    prompt = [{
        "role": "system",
        "content": f"{foundation_prompt} {data_shape_prompt}"
    }]
    
    return prompt


In [ ]:
def generate_content(client, prompt):
    """
    Fetches content from OpenAI based on an initial prompt and processes it to return a detailed JSON object.
    
    Args:
        client: An instance of the OpenAI client initialized with proper API credentials.
        prompt: A list containing a dictionary structured for OpenAI's input to generate text-based content.
    
    Returns:
        dict: A dictionary representing a JSON object with the keys 'story', 'scene_description', and 'player_options'.
    
    Raises:
        ValueError: If the response is incomplete due to length constraints or if expected keys are missing in the response.
    """
    # Fetch the response from OpenAI based on the provided prompt
    response = client.chat.completions.create(
        model=model_config['chat_model']['options'][model_config['chat_model']['selected']],
        messages=prompt,
        response_format={"type": "json_object"}  # Ensures response is formatted as JSON
    )

    # Check if the response was cut off because of reaching a length limit
    if response.choices[0].finish_reason != 'length':
        # Convert the JSON formatted string in the response to a Python dictionary
        content = json.loads(response.choices[0].message.content)
        
        # Define the set of keys expected in a valid response
        expected_keys = {'story', 'scene_description', 'player_options'}
        
        # Check if all expected keys are present in the response
        if not expected_keys <= content.keys():
            raise ValueError("Some expected keys are missing in the response")

        # Print the JSON content in a formatted way for better readability
        print(json.dumps(content, indent=4))
        
        # Return the processed content as a dictionary
        return content
    else:
        # Raise an error if the response was cut off
        raise ValueError("Response was cut off due to length limit.")



In [ ]:
# Generate random genre and associated story details for a text adventure game
story_data = select_random_genre()

# Print the selected story data for verification and review
print("Selected Story Data:", json.dumps(story_data, indent=4))


In [ ]:
import requests

def generate_audio(description: str, voice: str) -> str:
    """
    Generates an audio file from the provided text description using a specified voice model,
    and saves the audio to a file.

    Args:
        description (str): The text to be converted into speech.
        voice (str): The voice model to be used for speech synthesis.

    Returns:
        str: A message indicating whether the audio was successfully saved or if the synthesis failed.

    Raises:
        HTTPError: If the API request fails.
    """
    # API endpoint for the OpenAI audio synthesis
    url = "https://api.openai.com/v1/audio/speech"
    
    # Headers for the API request including the required Authorization token
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Data payload for the API request
    data = {
        "model": model_config['audio_model']['options'][model_config['audio_model']['selected']], 
        "input": description, 
        "voice": voice
    }
    
    # Post the data to the API endpoint
    response = requests.post(url, json=data, headers=headers)
    
    # Check if the response status code indicates success
    if response.status_code == 200:
        # Write the received audio content to a file
        with open('audio_output.mp3', 'wb') as f:
            f.write(response.content)
        return "Audio has been saved as audio_output.mp3"
    else:
        # If the status code is not 200, raise an exception with the status code and error message
        response.raise_for_status()



In [ ]:
def generate_image(description: str, artist: str) -> str:
    """
    Generates an image based on a description and artist style, intended for use in a text adventure game. The image is
    retrieved from the OpenAI image generation API and saved locally.

    Args:
        description (str): The scene description to be visualized.
        artist (str): The artistic style to emulate in the image creation.

    Returns:
        str: A message indicating whether the image was successfully saved or if there was an error.

    Raises:
        HTTPError: If the API request to generate or download the image fails.
    """
    # URL for the OpenAI image generation API
    url = "https://api.openai.com/v1/images/generations"
    
    # Headers for the API request including the required Authorization token
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Data payload for the API request
    data = {
        "model": model_config['image_model']['options'][model_config['image_model']['selected']],
        "prompt": f"You are {artist} creating this scene: {description} for a text adventure game. The image should be from the eyeline of the player and very true to the description. The painting should include all aspects of the description, and always include any mentioned paths or walkways. All pathways MUST be shown PROMINENTLY. No text should be included in the image.",
        "size": "1024x1024",
        "n": 1,
        "response_format": "url"
    }

    # Make the API call to generate the image
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        # Extract the URL of the generated image from the response
        image_url = response.json()['data'][0]['url']
        
        # Download the image from the provided URL
        image_response = requests.get(image_url)
        if image_response.status_code == 200:
            image_path = 'scene_image.png'
            with open(image_path, 'wb') as f:
                f.write(image_response.content)
            return f"Image has been saved as {image_path}"
        else:
            # If downloading the image fails, raise an HTTP error with the response
            image_response.raise_for_status()
    
    # If the image generation request fails, raise an HTTP error with the response
    response.raise_for_status()


In [ ]:
# Generate a narrative prompt using the selected story data from a random genre
generated_prompt = generate_prompt(story_data)

# Print the generated prompt to verify the structure and content
print(generated_prompt)


In [ ]:
# Generate a narrative content based on the structured prompt using the OpenAI client
scene = generate_content(client, generated_prompt)

# Using the generated story content to synthesize an audio file in the specified voice
generate_audio(scene['story'], story_data['voice'])

# Generate an image based on the scene description from the generated content, using the specified artist style
generate_image(scene['scene_description'], story_data['artist'])


In [ ]:
# Update the story_data with the story returned from the previous content generation
story_data['previous_scene'] = scene['story']

# Manually setting the player's move direction based on available options
# Assuming the "forward" direction is available and chosen
story_data['player_move'] = "forward"

# Regenerate a narrative prompt using the updated story data
generated_prompt = generate_prompt(story_data)

# Print the newly generated prompt to verify the updated structure and content
print(generated_prompt)


In [ ]:
def generate_content_and_media():
    # Generate narrative content based on the structured prompt
    scene = generate_content(client, generated_prompt)
    
    # Function to execute audio generation asynchronously
    def generate_audio_async():
        return generate_audio(scene['story'], story_data['voice'])
    
    # Function to execute image generation asynchronously
    def generate_image_async():
        return generate_image(scene['scene_description'], story_data['artist'])
    
    # Create a ThreadPoolExecutor to handle asynchronous tasks
    with ThreadPoolExecutor() as executor:
        # Submit audio and image generation tasks to the executor
        audio_future = executor.submit(generate_audio_async)
        image_future = executor.submit(generate_image_async)
        
        # Wait for both futures to complete and get their results
        audio_result = audio_future.result()
        image_result = image_future.result()

        # Optionally, print the results to verify
        print(audio_result)
        print(image_result)

# Call the function to execute the process
generate_content_and_media()


In [ ]:
# Update the story_data with the story returned from the previous content generation
story_data['previous_scene'] = scene['story']

# Manually setting the player's move direction based on available options
# Assuming the "forward" direction is available and chosen
story_data['player_move'] = "forward"

# Regenerate a narrative prompt using the updated story data
generated_prompt = generate_prompt(story_data)

# Print the newly generated prompt to verify the updated structure and content
print(generated_prompt)


In [ ]:
def generate_content_and_media():
    # Generate narrative content based on the structured prompt
    scene = generate_content(client, generated_prompt)
    
    # Function to execute audio generation asynchronously
    def generate_audio_async():
        return generate_audio(scene['story'], story_data['voice'])
    
    # Function to execute image generation asynchronously
    def generate_image_async():
        return generate_image(scene['scene_description'], story_data['artist'])
    
    # Create a ThreadPoolExecutor to handle asynchronous tasks
    with ThreadPoolExecutor() as executor:
        # Submit audio and image generation tasks to the executor
        audio_future = executor.submit(generate_audio_async)
        image_future = executor.submit(generate_image_async)
        
        # Wait for both futures to complete and get their results
        audio_result = audio_future.result()
        image_result = image_future.result()

        # Optionally, print the results to verify
        print(audio_result)
        print(image_result)

# Call the function to execute the process
generate_content_and_media()


In [ ]:
# Update the story_data with the story returned from the previous content generation
story_data['previous_scene'] = scene['story']

# Manually setting the player's move direction based on available options
# Assuming the "forward" direction is available and chosen
story_data['player_move'] = "forward"

# Regenerate a narrative prompt using the updated story data
generated_prompt = generate_prompt(story_data)

# Print the newly generated prompt to verify the updated structure and content
print(generated_prompt)


In [ ]:
def generate_content_and_media():
    # Generate narrative content based on the structured prompt
    scene = generate_content(client, generated_prompt)
    
    # Function to execute audio generation asynchronously
    def generate_audio_async():
        return generate_audio(scene['story'], story_data['voice'])
    
    # Function to execute image generation asynchronously
    def generate_image_async():
        return generate_image(scene['scene_description'], story_data['artist'])
    
    # Create a ThreadPoolExecutor to handle asynchronous tasks
    with ThreadPoolExecutor() as executor:
        # Submit audio and image generation tasks to the executor
        audio_future = executor.submit(generate_audio_async)
        image_future = executor.submit(generate_image_async)
        
        # Wait for both futures to complete and get their results
        audio_result = audio_future.result()
        image_result = image_future.result()

        # Optionally, print the results to verify
        print(audio_result)
        print(image_result)

# Call the function to execute the process
generate_content_and_media()
